In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
!pip install openai==0.28



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
import openai
import json
import random
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key=os.getenv("OPENAI_API_KEY")

In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below

def get_weather(lat,long):
	url = "https://api.open-meteo.com/v1/forecast"
	params = {
		"latitude": lat,
		"longitude": long,
		"current": "temperature_2m"
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]

	# Current values. The order of variables needs to be the same as requested.
	current = response.Current()
	current_temperature_2m = current.Variables(0).Value()

	return round(current_temperature_2m,1)

In [4]:
get_weather(28.6174,77.2090)

32.6

In [5]:
cities = [
    {"name": "San Francisco", "latitude": 37.7749, "longitude": 122.4194},
    {"name": "New York City", "latitude": 40.7128, "longitude": 74.0060},
    {"name": "Los Angeles", "latitude": 34.0522, "longitude": 118.2437},
    {"name": "Chicago", "latitude": 41.8781, "longitude": 87.6298},
    {"name": "Miami", "latitude": 25.7617, "longitude": 80.1918},
    {"name":"Delhi","latitude":28.6139,"longitude":77.2090},
    {"name":"Mumbai","latitude":19.0760,"longitude":72.8777}
]

def get_weather_data(city_name):
    # Find the city name from our city database
    city = next((c for c in cities if c["name"] == city_name), None)

    # If city is found, get the weather data using its coordinates
    if city:
        temp = get_weather(city['latitude'],city["longitude"])
        return f"Current Weather in {city['name']} is {temp}°C"
    else:
        return f"{city_name} Not found in Database :( )"

In [6]:
functions = [{
    "name": "get_weather_data",
    "description": "This function can get weather information for some of the cities",
    "parameters": {
        "type": "object",
        "properties": {
            "city_name": {
                "type": "string",
                "enum": ["San Francisco", "New York City", "Los Angeles", "Chicago", "Miami","Delhi"],
            },
        },
        "required": ["city_name"],
    },
}]
available_functions = {
    "get_weather_data": get_weather_data,
}

In [7]:
def converse(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    
    
    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        print('LLM calling the function =',function_name,"\n")
        function_response = function_to_call(
            city_name=function_args.get("city_name"),
        )


        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        print('Assistant: ', function_response)
        
    else: 
        content = response_message.get("content")
        print('Assistant: ', content)

In [8]:
messages = []
message_count = 0 
while message_count < 3: 
    user_input = input("User: ")
    user_message = {"role": "user", "content": user_input}
    messages.append(user_message)
    converse(messages)
    message_count += 1

Assistant:  Hi! How can I assist you today?
Assistant:  Delhi is located in northern India. It is the capital territory of India and is situated on the banks of the Yamuna River.
LLM calling the function = get_weather_data 

Assistant:  Current Weather in Delhi is 32.5°C
